Encoders:
 - https://huggingface.co/monsoon-nlp/hindi-bert
 - https://huggingface.co/flax-community/roberta-hindi
 - https://huggingface.co/vasista22/whisper-hindi-small
 
Decoders:
- https://huggingface.co/csebuetnlp/mT5_m2o_hindi_crossSum
- https://huggingface.co/docs/transformers/model_doc/openai-gpt
- https://huggingface.co/docs/transformers/model_doc/ctrl
- https://huggingface.co/docs/transformers/model_doc/gpt2

In [82]:
# from transformers import EncoderDecoderModel, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("monsoon-nlp/hindi-bert")
# model = EncoderDecoderModel.from_encoder_decoder_pretrained("monsoon-nlp/hindi-bert", "gpt2")

# model.config.decoder_start_token_id = tokenizer.cls_token_id
# model.config.pad_token_id = tokenizer.pad_token_id

# # a hindi sentence
# input_ids = tokenizer(
#     'भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष राहुल गांधी ने शनिवार को कहा कि भारतीय जनता पार्टी ने देश को एक दशक में वापस ले जाया है।',
#     return_tensors="pt",
# ).input_ids

# labels = tokenizer(
#     'राहुल गांधी ने कहा कि भारतीय जनता पार्टी ने देश को एक दशक में वापस ले जाया है।', return_tensors="pt",
# ).input_ids

# # the forward function automatically creates the correct decoder_input_ids
# outputs = model(input_ids=input_ids, labels=labels)

In [83]:
# outputs = model.generate(input_ids, decoder_start_token_id=model.config.decoder_start_token_id)
# outputs

In [84]:
# tokenizer2 = AutoTokenizer.from_pretrained("gpt2")
# tokenizer2.decode(outputs[0], skip_special_tokens=True, predict_with_generate=True)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_df = pd.read_csv('/mnt/disk1/sumdev/CourseWork/NLP/NLP-Project/Data/hindi_train.csv')
test_df = pd.read_csv('/mnt/disk1/sumdev/CourseWork/NLP/NLP-Project/Data/HindiNews_test.csv')
print(train_df.shape, test_df.shape)

train_df = train_df.dropna()
test_df = test_df.dropna()

train_val_split = 0.15
train_df, val_df = train_test_split(train_df, test_size=train_val_split, random_state=42)
print(train_df.shape, val_df.shape, test_df.shape)

(21225, 4) (3000, 3)
(18041, 4) (3184, 4) (3000, 3)


In [2]:
train_df.head()

,Id,Heading,Summary,Article
8713,hindi_2023_train_8713,प्रोडक्शन के लिए एम्ब्रेयर और सुखोई की बातचीत ...,india looks partner sukhoi embraer to manufact...,भारत सरकार दूर-दराज इलाकों से बेहतर एयर कनेक्ट...
19043,hindi_2022_9186,UP: भदोही में नशे में धुत युवक ने महात्मा गांध...,जिसके बाद कांग्रेस कार्यकर्ता एकत्रित होकर गां...,भदोहीः उत्तर प्रदेश के भदोही में बीती शनिवार र...
17257,hindi_2022_7400,भारत के इस क्षेत्र में नहीं है Coronavirus का ...,भारत में कोविड-19 के मामले 97.35 लाख के पार चल...,कोच्चि: भारत में कोविड-19 के मामले 97.35 लाख क...
3156,hindi_2023_train_3156,12वीं में पासिंग परसेंटेज के आधार पर परीक्षा म...,NEET 2024 - No Minimum Marks Needed For NEET. ...,12वीं के बाद मेडिकल की पढ़ाई करने वाले छात्रों ...
15583,hindi_2022_5726,"उत्तर प्रदेश में कोरोना वायरस से 17 और मौतें, ...",उत्‍तर प्रदेश में शुक्रवार को कोरोना वायरस संक...,लखनऊ: उत्‍तर प्रदेश में शुक्रवार को कोरोना वाय...


In [64]:
import torch
from transformers import EncoderDecoderModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("monsoon-nlp/hindi-bert")
model = EncoderDecoderModel.from_encoder_decoder_pretrained("monsoon-nlp/hindi-bert", "gpt2")

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

Some weights of ElectraForCausalLM were not initialized from the model checkpoint at monsoon-nlp/hindi-bert and are newly initialized: ['electra.encoder.layer.0.crossattention.output.LayerNorm.bias', 'electra.encoder.layer.0.crossattention.output.LayerNorm.weight', 'electra.encoder.layer.0.crossattention.output.dense.bias', 'electra.encoder.layer.0.crossattention.output.dense.weight', 'electra.encoder.layer.0.crossattention.self.key.bias', 'electra.encoder.layer.0.crossattention.self.key.weight', 'electra.encoder.layer.0.crossattention.self.query.bias', 'electra.encoder.layer.0.crossattention.self.query.weight', 'electra.encoder.layer.0.crossattention.self.value.bias', 'electra.encoder.layer.0.crossattention.self.value.weight', 'electra.encoder.layer.1.crossattention.output.LayerNorm.bias', 'electra.encoder.layer.1.crossattention.output.LayerNorm.weight', 'electra.encoder.layer.1.crossattention.output.dense.bias', 'electra.encoder.layer.1.crossattention.output.dense.weight', 'electra.e

In [59]:
import torch.nn as nn
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 3

In [60]:
def train(model, tokenizer, train_df, optimizer):
    model.train()
    total_loss = 0
    batch = 0
    batch_loss = 0
    batch_size = 1000
    for i, row in train_df.iterrows():
        input_ids = tokenizer(row['Heading'], return_tensors="pt").input_ids.to(device)
        labels = tokenizer(row['Summary'], return_tensors="pt").input_ids.to(device)
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        batch_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        batch += 1
        if batch % batch_size == 0:
            print(f"Batch {batch} Loss: {batch_loss / batch_size}")
            batch_loss = 0
    print(f"Training Loss: {total_loss / len(train_df)}")
        

def evaluate(model, tokenizer, val_df):
    model.eval()
    total_loss = 0
    for i, row in val_df.iterrows():
        input_ids = tokenizer(row['Heading'], return_tensors="pt").input_ids.to(device)
        labels = tokenizer(row['Summary'], return_tensors="pt").input_ids.to(device)

        outputs = model(input_ids=input_ids, labels=labels)
        total_loss += outputs.loss.item()
    print(f"Validation Loss: {total_loss / len(val_df)}")


for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")
    train(model, tokenizer, train_df, optimizer)
    evaluate(model, tokenizer, val_df)

Epoch 1


/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:616: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:636: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWa

Batch 1000 Loss: 7.803432563304901
Batch 2000 Loss: 7.540177937030792
Batch 3000 Loss: 7.524164014339447
Batch 4000 Loss: 7.456521791934967
Batch 5000 Loss: 7.468214980602264
Batch 6000 Loss: 7.430041268348694
Batch 7000 Loss: 7.421582843780517
Batch 8000 Loss: 7.435788701534271
Batch 9000 Loss: 7.451207971572876
Batch 10000 Loss: 7.413762616634369
Batch 11000 Loss: 7.4254929637908935
Batch 12000 Loss: 7.4491350345611576
Batch 13000 Loss: 7.52306252002716
Batch 14000 Loss: 7.412403644084931
Batch 15000 Loss: 7.413734418392181
Batch 16000 Loss: 7.399366858005524
Batch 17000 Loss: 7.397126619815826
Batch 18000 Loss: 7.415912605762482
Training Loss: 7.465806795312338
Validation Loss: 7.419983010046447
Epoch 2
Batch 1000 Loss: 7.290245727539062
Batch 2000 Loss: 7.332596723079681
Batch 3000 Loss: 7.371773745059967
Batch 4000 Loss: 7.33793325138092
Batch 5000 Loss: 7.3714003348350525
Batch 6000 Loss: 7.336872177600861
Batch 7000 Loss: 7.335406280517578
Batch 8000 Loss: 7.344756159305573
Batc

KeyboardInterrupt: 

In [81]:
text = test_df['Article'][6]
print(test_df['id'][6])
print(text)
input_ids = tokenizer(text, return_tensors="pt", truncation=True).input_ids.to(device)
print(input_ids.shape)

outputs = model.generate(input_ids, decoder_start_token_id=model.config.decoder_start_token_id)
print(outputs)
# tokenizer2
summary = tokenizer.decode(outputs[0], skip_special_tokens=True, predict_with_generate=True)
print("Summary:", summary)

HindiNews_test_6
साल-दर-साल बढ़ती गर्मी के चलते देश की खेती, अर्थव्यवस्था और लोगों के स्वास्थ्य पर बहुत दबाव पड़ रहा है। वहीं, जलवायु परिवर्तन की वजह से गरीबी, असमानता और बीमारियों को खत्म करने के देश की कोशिशें भी नाकाम हो रही हैं। यह खुलासा कैंब्रिज यूनिवर्सिटी की एक नई स्टडी में हुआ है। इसे स्कॉलर्स की एक टीम ने पब्लिश किया है।
टीम के लीडर रमित देबनाथ ने कहा कि लू के चलते 1992 से अब तक देश में 24 हजार से ज्यादा मौतें दर्ज हुई हैं। गर्मी के चलते वायु प्रदूषण बढ़ गया है और उत्तर भारत के पहाड़ों में मौजूद ग्लेशियर पिघलने लगे हैं। देश में जनवरी से अक्टूबर तक तकरीबन हर रोज भीषण मौसम देखने को मिल रहा है, जिसकी वजह से देश को एक साथ कई सारी मौसमी परेशानियों का सामना करना पड़ सकता है।
देश का 90% इलाका हीट जोन बना 
देबनाथ ने रॉयटर्स को बताया कि लगभग भारत के कुल एरिया में से 90% गंभीर हीट जोन बन चुके हैं और ये इस गर्मी से निपटने से के लिए तैयार नहीं हैं। उन्होंने कहा कि भारत ने लू को अपने डिजास्टर रिलीफ पैकेज में शामिल किया है, लेकिन इन योजनाओं की रफ्तार को तेज करने की जरूरत है।
भीषण गर्मी के चलते 